In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr
from tqdm.notebook import tqdm

from cellpack_analysis.utilities.PILR_tools import get_cellid_ch_seed_from_fname
from cellpack_analysis.utilities.plotting_tools import read_PILR_from_tiff


/allen/aics/animated-cell/Saurabh/Softwares/miniconda3/envs/cellpack_analysis/lib/python3.9/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
/allen/aics/animated-cell/Saurabh/Softwares/miniconda3/envs/cellpack_analysis/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.17) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
STRUCTURE_ID = "SLC25A17"
STRUCTURE_NAME = "peroxisome"
FOLDER_ID = "RSV_8_shapes_all_reps"

In [3]:
results_folder = Path(f"/allen/aics/animated-cell/Saurabh/cellpack-analysis/results/{STRUCTURE_ID}/{FOLDER_ID}/")
data_folder = Path("/allen/aics/animated-cell/Saurabh/cellpack-analysis/data")

#### Load individual PILR dict

In [ ]:
with open(results_folder / "individual_PILR.json") as f:
    individual_PILR_dict = json.load(f)

In [ ]:
pilr_dict = {}
for ch_name in channels_to_use:
    pilr_dict[ch_name] = np.array(individual_PILR_dict[ch_name])

In [ ]:
print(individual_PILR_dict.keys())

#### Set channels to use

In [5]:
channels_to_use = [
    "SLC25A17",
    "nucleus_moderate",
]
raw_image_channel = "SLC25A17"
analyze_channel = "nucleus_moderate"

#### Load PILRs from images (this takes a while)

In [6]:
pilr_image_path = results_folder / "individual_PILR/"

In [7]:
shape_pilr_dict = {}
ct = 0
tuple_list = []
for ch in channels_to_use:
    shape_pilr_dict[ch] = {}
    print(f"Processing {ch}")
    for file in tqdm(pilr_image_path.glob(f"*{ch}*")):
        if ch == raw_image_channel:
            cellid = file.name.split("_")[1]
            seed_name = "0"
        else:
            cellid, _, seed_name = get_cellid_ch_seed_from_fname(file.name)
            tuple_list.append((ch, cellid, seed_name))
        if cellid not in shape_pilr_dict[ch]:
            shape_pilr_dict[ch][cellid] = {}
        shape_pilr_dict[ch][cellid][seed_name] = read_PILR_from_tiff(file)
        


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
len(tuple_list)

2440

#### set peroxisome stereotypy and std

In [ ]:
peroxisome_stereotypy = 2.627e-3
peroxisome_error = 8.395e-3

#### create dataframe for individual correlations

In [ ]:
shape_cellids = list(shape_pilr_dict[analyze_channel].keys())
print(shape_cellids)

In [ ]:
# create multiindex dataframe with channel, cellid and seed as indices and columns
df_corr = pd.DataFrame(
    index=pd.MultiIndex.from_tuples(
        tuple_list,
        names=["channel", "cellid", "seed"],
    ),
    columns=[f"{raw_image_channel}_corr"],
)

In [ ]:
for level in [1, 2]:
    df_corr.index = df_corr.index.set_levels(df_corr.index.levels[level].astype(str), level=level)

In [ ]:
df_corr

#### Calculate pairwise correlations

In [ ]:
# get pairwise correlations
for cellid, cellid_dict in shape_pilr_dict[analyze_channel].items():
    print(f"processing cellid {cellid}")
    for seed_name, seed_pilr in tqdm(cellid_dict.items(), total=len(cellid_dict)):
        analyze_pilr = seed_pilr[(seed_pilr.shape[0] // 2):, :]
        raw_pilr = shape_pilr_dict[raw_image_channel][cellid]["0"][(seed_pilr.shape[0] // 2):, :]
        df_corr.loc[
            (analyze_channel, cellid, seed_name),
            f"{raw_image_channel}_corr",
        ] = pearsonr(
            analyze_pilr.flatten(),
            raw_pilr.flatten(),
        )[0]
    df_corr.to_csv(results_folder / "individual_PILR_correlation.csv")
df_corr.to_csv(results_folder / "individual_PILR_correlation.csv")

In [ ]:
df_corr

#### Make violinplot across shapes

In [ ]:
# plot average and std on barplots
cellid_vals = df_corr.index.get_level_values("cellid").unique()
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=300)
# set font size
xvals = np.arange(len(cellid_vals))
cellid_labels = [f"Shape {i + 1}" for i in xvals]
ax = sns.violinplot(
    data=df_corr,
    x="cellid",
    y=f"{raw_image_channel}_corr",
    ax=ax,
)
ax.set_xticklabels([f"Shape {i+1}" for i in range(8)] , rotation=45)
ax.set_ylabel("Correlation with SLC25A17 data\n in the same shape")
ax.set_xlabel("Shape IDs")
ax.axhline(peroxisome_stereotypy, color="r", ls="--", alpha=0.5)
ax.axhspan(peroxisome_stereotypy - peroxisome_error, peroxisome_stereotypy + peroxisome_error, color="r", alpha=0.1, zorder=-1)
ax.set_title("n=305 replicates for each shape")
plt.tight_layout()
plt.show()
fig.savefig(results_folder / "replicate_stereotypy_comparison.png", dpi=300)